# Emoji generator using GAN.
Addapted from Keras Minimal Viable GAN

In [12]:
import csv
import imageio
import os
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image
from tqdm import tqdm  # performance timing

# Building on Keras
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import fashion_mnist
from keras.optimizers import Adam
from keras import initializers
from skimage import color

np.random.seed(10)
random_dim = 100


In [13]:
emojis = np.array([imageio.imread(os.path.join('final_emoji/', emoji)) 
                   for emoji in os.listdir('final_emoji/')])


In [14]:
emojis = emojis / 255

In [15]:
emojis.shape

(976, 36, 36, 3)

In [16]:
emojis = np.array([emoji.flatten() for emoji in emojis])

In [17]:
emojis.shape

(976, 3888)

In [18]:
def get_discriminator(optimizer):
    discriminator = Sequential()
    discriminator.add(Dense(
        1024, input_dim=3888,
        kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
 
    discriminator.add(Dense(1024))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
 
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
 
    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return discriminator

def get_generator(optimizer):
    generator = Sequential()
    generator.add(Dense(
        256, input_dim=random_dim,
        kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))
 
    generator.add(Dense(524))
    generator.add(LeakyReLU(0.2))
 
    generator.add(Dense(1048))
    generator.add(LeakyReLU(0.2))
 
    generator.add(Dense(3888, activation='sigmoid'))
    generator.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return generator

def get_gan_network(discriminator, random_dim, generator, optimizer):
    # We initially set trainable to False since we only want to train either the 
    # generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = Input(shape=(random_dim,))
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real/not)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan



In [19]:
 def plot_generated_images(epoch, generator, examples=100, dim=(10, 10),
                           figsize=(15, 15)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 36, 36, 3)
 
    plt.figure(figsize=figsize)
    for i in range(10):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow((generated_images[i] * 255).astype(np.uint8), interpolation='nearest')
        plt.axis('off')
        # plt.title(f'epoch: {epoch}')
    plt.tight_layout()
    if epoch == 25000:
        plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [20]:
def train(x_train, epochs=1, batch_size=128):
    # Get the training and testing data
    # x_train, y_train, x_test, y_test = load_minst_data()
    # Split the training data into batches of size 128
    batch_count = x_train.shape[0] // batch_size
 
    # Build our GAN netowrk
    adam = Adam(lr=0.0002, beta_1=0.5)
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)
    
    # Pre-train generator and discriminator.
    for e in tqdm(range(1, 10000 + 1)):
        # Generator.
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        image_batch = x_train[np.random.randint(0, x_train.shape[0],
                                               size=batch_size).flatten()]
        generator.train_on_batch(noise, image_batch)
        
        # Discriminator.
        noise = np.random.normal(0, 1, size=[batch_size, 3888])
        image_batch = x_train[np.random.randint(0, x_train.shape[0],
                                                size=batch_size)]
        y_dis = np.zeros(2 * batch_size)
        y_dis[:batch_size] = 0.9
        X = np.concatenate([image_batch, noise])
        discriminator.trainable = True
        discriminator.train_on_batch(X, y_dis)
    
    
 
    for e in tqdm(range(1, epochs + 1)):
#         print (f"{'-' * 15}, 'Epoch {e}, {'-' * 15}")
        for _ in range(batch_count):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            image_batch = x_train[np.random.randint(0, x_train.shape[0],
                                                    size=batch_size)]
 
            # Generate fake images
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])
 
            # Labels for generated and real data
            y_dis = np.zeros(2*batch_size)
            # One-sided label smoothing
            y_dis[:batch_size] = 0.9
 
            # Train discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)
 
            # Train generator
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)
 
        if e % 1000 == 0:
            plot_generated_images(e, generator)
    return generator

In [21]:
epochs = 500000

In [ ]:
generator = train(emojis, epochs)


100%|██████████| 10000/10000 [06:20<00:00, 26.26it/s]

  4%|▎         | 18509/500000 [46:15<20:22:23,  6.56it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)


  4%|▍         | 19991/500000 [49:58<20:09:37,  6.61it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)


  4%|▍         | 22213/500000 [55:31<20:05:51,  6.60it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid cra

In [ ]:
plot_generated_images(epochs, generator)

In [ ]:
noise = np.random.normal(0, 1, size=[1, random_dim])
x = generator.predict(noise)
fig = plt.figure(dpi=15)
ax = fig.gca()
ax.imshow((x.reshape(36, 36, 3) * 255).astype(np.uint8), interpolation='nearest')
ax.axis('off')


In [ ]:
# fig.savefig('gan_disappoint.jpg', format='jpg')